# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 1

<font color='orange'>Agente aspirador en una grilla de NxN</font>

En el presente notebook se tiene la codificación de un agente aspirador (*VacuumAgent*) así como de un entorno tipo grilla configurable (*room*). Se busca que el agente interactúe con su entorno para limpiarlo.

## Clase <b>Thing</b>

  Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>.

In [0]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

## Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [0]:
import collections

class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

## Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [0]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade un objeto Thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)
    
    def delete_thing(self, thing):
        """Elimina un objeto Thing al entorno en la localizacion location
        Si thing es un programa de agente, igual lo remueve.
        Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        return [thing for thing in self.things
                if thing.location == location and isinstance(thing, tclass)]

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

## Clase XYEnvironment

In [0]:
class XYEnvironment(Environment):
    """This class is for environments on a 2D plane, with locations
    labelled by (x, y) points, either discrete or continuous.

    Agents perceive things within a radius. Each agent in the
    environment has a .location slot which should be a location such
    as (0, 1), and a .holding slot, which should be a list of things
    that are held."""

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width, self.height)

    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """Return all things within radius of location."""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        return [(thing, radius2 - distance_squared(location, thing.location))
                for thing in self.things if distance_squared(
                                                location, thing.location) <= radius2]

    def percept(self, agent):
        """By default, agent perceives things within a default radius."""
        return self.things_near(agent.location)

    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
#         elif action == 'Grab':
#             things = [thing for thing in self.list_things_at(agent.location)
#                     if agent.can_grab(thing)]
#             if things:
#                 agent.holding.append(things[0])
        elif action == 'Release':
            if agent.holding:
                agent.holding.pop()

    def default_location(self, thing):
        return (random.choice(self.width), random.choice(self.height))

    def move_to(self, thing, destination):
        """Move a thing to a new location. Returns True on success or False if there is an Obstacle.
        If thing is holding anything, they move with him."""
        thing.bump = self.some_things_at(destination, Obstacle)
        if not thing.bump:
            thing.location = destination
            for o in self.observers:
                o.thing_moved(thing)
            for t in thing.holding:
                self.delete_thing(t)
                self.add_thing(t, destination)
                t.location = destination
        return thing.bump

    def add_thing(self, thing, location=(1, 1), exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if (self.is_inbounds(location)):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            super().add_thing(thing, location)

    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while(location == exclude):
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

    def delete_thing(self, thing):
        """Deletes thing, and everything it is holding (if thing is an agent)"""
        if isinstance(thing, Agent):
            for obj in thing.holding:
                super().delete_thing(obj)
                for obs in self.observers:
                    obs.thing_deleted(obj)

        super().delete_thing(thing)
        for obs in self.observers:
            obs.thing_deleted(thing)

    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        for x in range(self.width):
            self.add_thing(Wall(), (x, 0))
            self.add_thing(Wall(), (x, self.height - 1))
        for y in range(1, self.height-1):
            self.add_thing(Wall(), (0, y))
            self.add_thing(Wall(), (self.width - 1, y))

        # Updates iteration start and end (with walls).
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width - 1, self.height - 1)

    def add_observer(self, observer):
        """Adds an observer to the list of observers.
        An observer is typically an EnvGUI.

        Each observer is notified of changes in move_to and add_thing,
        by calling the observer's methods thing_moved(thing)
        and thing_added(thing, loc)."""
        self.observers.append(observer)

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)
# ______________________________________________________________________________

## Clases Obstacle and Wall

In [0]:
class Obstacle(Thing):
    """Something that can cause a bump, preventing an agent from
    moving into the same square it's in."""
    pass

class Wall(Obstacle):
    pass

## Clase GraphicEnvironment

In [45]:
#@title Instalacion de `ipythonblocks`
!pip install ipythonblocks

In [0]:
from ipythonblocks import BlockGrid
from IPython.display import HTML, display
from time import sleep
class GraphicEnvironment(XYEnvironment):
    def __init__(self, width=10, height=10, color={}, boundary=True, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""
        super().__init__(width, height)
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at([x, y]))
            result.append(row)
        return result

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=10, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)

    def update(self, delay=1):
        sleep(delay)
        if self.visible:
            self.conceal()
            self.reveal()
        else:
            self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

## Clase VacuumEnvironment

In [0]:
from IPython.core.debugger import set_trace

In [0]:
loc_A, loc_B, loc_C, loc_D = [0, 0], [1, 0], [0, 1], [1, 1]   # The 4 locations for the Vacuum world 2x2

import random
class VacuumEnvironment(GraphicEnvironment):
    def __init__(self, width, height, color, autoDirt=False, boundary=True, display=False, verbose=False):
        super().__init__(width, height, color, boundary, display)
        self.autoDirt = autoDirt
        self.verbose = verbose

    def thing_classes(self):
        return [VacuumBot]

    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def ValidarVal(self,a, maxVal):
        if(a<0):
          return 0
        elif(a>=maxVal):
          return maxVal-1
        else:
          return a
    
    def execute_action(self, agent, action):
        """Implementa el MAPA DE TRANSICION: Cambia la posicion del agente y/o el estado de la posicion; 
        Cada aspiracion (accion 'Suck') en una localizacion Dirt provoca un aumento de desempeño en 10 unidades.
        Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
        
        if action == 'Right':#aumenta en 1 el primer elemento de location
            newCord = self.ValidarVal(agent.location[0]+1, self.width)
            agent.location = [newCord, agent.location[1]]
            agent.performance -= 1
        elif action == 'Left':#disminuye en 1 el primer elemento de location
            newCord = self.ValidarVal(agent.location[0]-1, self.width)
            agent.location = [newCord, agent.location[1]]
            agent.performance -= 1
        elif action == 'Up':#disminuye en 1 el segundo elemento de location
          newCord = self.ValidarVal(agent.location[1]-1, self.width)
          agent.location = [agent.location[0], newCord]
          agent.performance -= 1
        elif action == 'Down':#aumenta en 1 el segundo elemento de location
          newCord = self.ValidarVal(agent.location[1]+1, self.width)
          agent.location = [agent.location[0], newCord]
          agent.performance -= 1
        elif action == 'Suck':
          items = self.list_things_at(agent.location, tclass=Dirt)
          if len(items) != 0:
            if(self.verbose): print("Quiero limpiar las sucias", items, "en",agent.location)
            for i in items:
              if agent.clean(i): #Have the robot clean the first item
                print('{} cleaned {} at location: {}'
                      .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                self.delete_thing(i) #Delete it from the Room after.
          agent.performance += 10

        #Si es que self.autoDirt está activado, aleatoriamente va a ensuciar las casillas vacias
        if(self.autoDirt==True):
          world = self.get_world()
          for x in range(0, len(world)):
            for y in range(0, len(world[x])):
              if (  len( self.list_things_at([x, y]) ) == 0  ):
                if(self.verbose): print("Estoy queriendo ensuciar en:",x,y)
                dirt = Dirt()
                numMovil = random.choice(list(range(0,100)))
                if (numMovil < 2):#probabilidad de 2% de ensuciar cada casilla
                  if(self.verbose): print("Si he ensuciado en:",x,y)
                  self.add_thing(dirt, [x, y])

    def default_location(self, thing):
        """Devuelve una posicion aleatoria para la posición inicial de la aspiradora."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

## Clase Dirt

In [0]:
class Dirt(Thing):
    pass

In [50]:
dirt = Dirt()
id(dirt)

139998800211080

In [51]:
dirt = Dirt()
id(dirt)

139998800211304

## Clase VacuumBot

In [0]:
class VacuumBot(Agent):        
    def clean(self, thing):
        '''Retorna True si es que la limpieza fue satisfactoria y False en otros casos'''
        if isinstance(thing, Dirt):
            return True
        return False

## Definición de programas de agentes

In [0]:
def program(percepts):
  #Si alguna de mis percepciones es un Dirt, debo limpiar
  #Puede que mi primera percepción no sea un Dirt!
  for pp in percepts:
    if isinstance(pp, Dirt):
      return 'Suck'

  #si no está sucio, entonces allí está el agente. Reflexivamente, le asigno una accion para cada posicion
  for pp in percepts:      
    if pp.location == loc_A:
      return 'Right'
    elif pp.location == loc_B:
      return 'Down'
    elif pp.location == loc_C:
      return 'Up'
    elif pp.location == loc_D:
      return 'Left'

In [0]:
class Dirt(Thing):
    pass

In [55]:
dirt = Dirt()
id(dirt)

139998808947400

In [56]:
dirt = Dirt()
id(dirt)

139998808947960

## Creamos y probamos el entorno

In [0]:
room = VacuumEnvironment(2,2, color={'VacuumBot': (200,0,0), 'Dirt': (133, 122, 111)}, autoDirt=False, verbose=False) # room width is set to 2, and height to 2
vacuumBot = VacuumBot(program)
dirt = Dirt()
moredirt = Dirt()
room.add_thing(dirt, loc_A)
room.add_thing(moredirt, loc_C)

In [58]:
room.get_world()

[[[<__main__.Dirt at 0x7f5402bd4240>], [<__main__.Dirt at 0x7f5402bd4e48>]],
 [[], []]]

In [0]:
room.add_thing(vacuumBot, location=loc_C)

In [0]:
vacuumBot2 = VacuumBot(program)

In [0]:
room.add_thing(vacuumBot2, location=loc_B)

In [62]:
room.get_world()

[[[<__main__.Dirt at 0x7f5402bd4240>],
  [<__main__.Dirt at 0x7f5402bd4e48>, <__main__.VacuumBot at 0x7f5402bd4c88>]],
 [[<__main__.VacuumBot at 0x7f5402bd4b70>], []]]

In [0]:
room.autoDirt=True

In [64]:
room.run(20)

,
,


__main__.VacuumBot object at 0x7f5402bd4c88 cleaned __main__.Dirt object at 0x7f5402bd4e48 at location: [0, 1]


,
,


,
,


__main__.VacuumBot object at 0x7f5402bd4c88 cleaned __main__.Dirt object at 0x7f5402bd4240 at location: [0, 0]


...

,
,


,
,


,
,


,
,


,
,


In [65]:
room.get_world()

[[[<__main__.VacuumBot at 0x7f5402bd4c88>,
  []],
 [[], []]]

In [66]:
room.get_world()[0]

[[<__main__.VacuumBot at 0x7f5402bd4c88>,
 []]

In [67]:
room.get_world()[1]

[[], []]

In [68]:
vacuumBot.performance, vacuumBot2.performance

(13, -9)

## Para desarrollar

Crear un entorno `room` de 4x4 de con dos agentes que se muevan de manera reflexiva. Se busca que puedan operar de forma autónoma y garanticen que todas las casillas del room se lleguen a limpiar constantemente. Activar la generación aleatoria de `dirt` en el entorno y modificarlo para que cada casilla tenga una probabilidad del 2% de ensuciarse.



In [0]:
def program_44_A(percepts):
  #Si alguna de mis percepciones es un Dirt, debo limpiar
  #Puede que mi primera percepción no sea un Dirt!
  for pp in percepts:
    if isinstance(pp, Dirt):
      return 'Suck'

  #si no está sucio, entonces allí está el agente. Reflexivamente, le asigno una accion para cada posicion
  for pp in percepts:      
    if pp.location in [[0,0],[1,0],[2,0]]:
      return 'Right'
    elif pp.location in [[1,1],[2,1],[3,1]]:
      return 'Left'
    elif pp.location == [3,0]:
      return 'Down'
    elif pp.location == [0,1]:
      return 'Up'

In [0]:
def program_44_B(percepts):
  #Si alguna de mis percepciones es un Dirt, debo limpiar
  #Puede que mi primera percepción no sea un Dirt!
  for pp in percepts:
    if isinstance(pp, Dirt):
      return 'Suck'

  #si no está sucio, entonces allí está el agente. Reflexivamente, le asigno una accion para cada posicion
  for pp in percepts:      
    if pp.location in [[0,3],[1,3],[2,3]]:
      return 'Right'
    elif pp.location in [[1,2],[2,2],[3,2]]:
      return 'Left'
    elif pp.location == [0,2]:
      return 'Down'
    elif pp.location == [3,3]:
      return 'Up'

In [0]:
room = VacuumEnvironment(4,4, color={'VacuumBot': (200,0,0), 'Dirt': (133, 122, 111)}, autoDirt=True, verbose=False) # room width is set to 2, and height to 2
vacuumBot_A = VacuumBot(program_44_A)
vacuumBot_B = VacuumBot(program_44_B)
dirt = Dirt()
moredirt = Dirt()
room.add_thing(dirt, loc_A)
room.add_thing(moredirt, loc_C)

In [0]:
room.add_thing(vacuumBot_A, location=[0,0])
room.add_thing(vacuumBot_B, location=[3,3])

In [73]:
room.get_world()

[[[<__main__.Dirt at 0x7f540276aeb8>, <__main__.VacuumBot at 0x7f540276a550>],
  [],
  []],
 [[], [], [], []],
 [[], [], [], []],
 [[], [], [], [<__main__.VacuumBot at 0x7f540276ac50>]]]

In [74]:
room.run(20)

,,,
,,,
,,,
,,,


__main__.VacuumBot object at 0x7f540276a550 cleaned __main__.Dirt object at 0x7f540276aeb8 at location: [0, 0]


,,,
,,,
,,,
,,,


,,,
,,,
,,,
,,,


__main__.VacuumBot object at 0x7f540276a550 cleaned __main__.Dirt object at 0x7f540276a400 at location: [1, 0]


...

__main__.VacuumBot object at 0x7f540276ac50 cleaned __main__.Dirt object at 0x7f540276af60 at location: [1, 3]


,,,
,,,
,,,
,,,


,,,
,,,
,,,
,,,


__main__.VacuumBot object at 0x7f540276a550 cleaned __main__.Dirt object at 0x7f540276a358 at location: [1, 1]
__main__.VacuumBot object at 0x7f540276ac50 cleaned __main__.Dirt object at 0x7f540276a3c8 at location: [2, 3]


,,,
,,,
,,,
,,,
